In [25]:

url = "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040"

# START CODE

In [26]:
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms 

import json
import cv2
import numpy as np

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Create a Torch Dataset

In [28]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file,"r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item=  self.data[idx]
        img_path = f"{self.img_dir}/{item['id']}.png"
        img = cv2.imread(img_path)
        
        if img is None:
            raise FileNotFoundError(f"Image not found: {img_path}")
        
        
        h,w = img.shape[:2]
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten() # Aplanar para convertir de 2D a 1D
        kps = kps.astype(np.float32)
        
        kps[::2] *= 224.0 / w # Adjust x coordinates (Lo hemos modificado)
        kps[1::2] *= 224.0 / h # Adjust y coordinates (lo hemos modificado)
        
        return img, kps
        


In [29]:
train_dataset = KeypointsDataset("./tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_train.json")
val_dataset = KeypointsDataset("./tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

## Create a model

In [30]:
model = models.resnet50(pretrained=True)  #Bajar preentrenada CNN , como no está entrenado para buscar los puntos, tendremos que reentrenarlo la última capa para eso. 
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # 14 puntos y cada punto tiene X e Y // Replace the last layer

C:\Users\caos_\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\caos_\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
#Move the model to device (GPU in our case)
model = model.to(device)

## Train the model

In [32]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [33]:
epochs = 20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        pred = model(imgs)
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward() #realizar el backpropagation
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Step {i+1} / {len(train_loader)}, Loss: {loss.item()}")

Epoch 1/20, Step 1 / 829, Loss: 15100.4892578125
Epoch 1/20, Step 11 / 829, Loss: 14752.4697265625
Epoch 1/20, Step 21 / 829, Loss: 15416.134765625
Epoch 1/20, Step 31 / 829, Loss: 13887.2568359375
Epoch 1/20, Step 41 / 829, Loss: 13452.814453125
Epoch 1/20, Step 51 / 829, Loss: 13135.7216796875
Epoch 1/20, Step 61 / 829, Loss: 12862.646484375
Epoch 1/20, Step 71 / 829, Loss: 12516.6103515625
Epoch 1/20, Step 81 / 829, Loss: 12171.5244140625
Epoch 1/20, Step 91 / 829, Loss: 11385.634765625
Epoch 1/20, Step 101 / 829, Loss: 11045.607421875
Epoch 1/20, Step 111 / 829, Loss: 10397.7392578125
Epoch 1/20, Step 121 / 829, Loss: 9260.8583984375
Epoch 1/20, Step 131 / 829, Loss: 9887.9375
Epoch 1/20, Step 141 / 829, Loss: 9587.33203125
Epoch 1/20, Step 151 / 829, Loss: 9921.955078125
Epoch 1/20, Step 161 / 829, Loss: 9088.8828125
Epoch 1/20, Step 171 / 829, Loss: 8479.0224609375
Epoch 1/20, Step 181 / 829, Loss: 7898.662109375
Epoch 1/20, Step 191 / 829, Loss: 8146.58203125
Epoch 1/20, Step 20

In [35]:
torch.save(model.state_dict(), "keypoints_model.pth")